# Testing Facebook prophet

In [1]:
# library
from tqdm import tqdm
import pandas as pd
from prophet import Prophet
import yfinance as yf
from datetime import datetime, timedelta
import plotly.express as px
import numpy as np

## 1. Acquire and preprocess data

### Data Retrieval from Yahoo finance

In [2]:
# Time periods
now = datetime.now()

# Period of analysis
ago = now - timedelta(days=730)

# Designating the Ticker (BTC)
crypto = yf.Ticker("ETH-USD")

# Getting price history
df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")

# Handling missing data from yahoo finance
df = df.reindex(
    [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
    fill_value=None
).fillna(method='ffill')

### Data processing

In [3]:
# Getting the N Day Moving Average and rounding the values
df['MA'] = df[['Open']].rolling(window=5).mean().apply(lambda x: round(x, 2))

# Dropping the NaNs
df.dropna(inplace=True)

### Formatting DF for FB Prophet

In [4]:
# Formatted
df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
df

,ds,Open,High,Low,Close,Volume,Dividends,Stock Splits,y
0,2021-02-23,1781.409058,1781.409058,1378.840942,1570.203979,52029864713,0,0,1906.88
1,2021-02-24,1571.476440,1710.983765,1511.018921,1626.575684,31329000537,0,0,1833.40
2,2021-02-25,1625.393921,1670.224121,1465.058960,1475.703735,24481681873,0,0,1766.50
3,2021-02-26,1478.653320,1559.028931,1407.979248,1446.033691,31435997881,0,0,1678.50
4,2021-02-27,1446.929443,1524.932373,1433.786987,1459.973145,20742103233,0,0,1580.77
...,...,...,...,...,...,...,...,...,...
718,2023-02-11,1514.941895,1541.334717,1510.146973,1539.926758,5064007416,0,0,1600.11
719,2023-02-12,1539.776001,1545.554199,1501.462036,1515.033691,5319698775,0,0,1584.86
720,2023-02-13,1514.916992,1524.731445,1470.023926,1507.165894,9363855114,0,0,1553.46
721,2023-02-14,1506.992065,1562.034302,1497.194458,1556.875122,9249575045,0,0,1524.64


## 2. Training and Predicting with the model

### Fitting to FB Prophet

In [5]:
# Setting up Prophet
m = Prophet(
    daily_seasonality=True, 
    yearly_seasonality=True, 
    weekly_seasonality=True
)

# Fitting to the prices
m.fit(df[['ds', 'y']])

20:02:45 - cmdstanpy - INFO - Chain [1] start processing
20:02:46 - cmdstanpy - INFO - Chain [1] done processing


### Getting Future Dates for Forecast

In [6]:
# Future DF of 3 months
future = m.make_future_dataframe(periods=90)

### Forecasting Crypto Prices

In [7]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2021-02-23,1393.431765,1074.819264,1725.994441
1,2021-02-24,1397.674638,1060.229342,1729.186495
2,2021-02-25,1404.816536,1078.962183,1729.568615
3,2021-02-26,1408.512073,1065.903892,1747.656471
4,2021-02-27,1407.592823,1075.277673,1744.292152
...,...,...,...,...
808,2023-05-12,2232.347167,1810.925862,2636.919545
809,2023-05-13,2212.731050,1801.386027,2596.557108
810,2023-05-14,2190.450894,1751.963952,2621.576836
811,2023-05-15,2164.838763,1770.049740,2576.889436


## 3. Visualising Forecast

In [20]:
# Visual DF
vis_df = df[['ds','Open']].append(
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
).rename(columns={'yhat': 'Prediction'})

# Visualizing results
fig = px.line(
    vis_df,
    x='ds',
    y=['Open', 'Prediction', 'yhat_lower', 'yhat_upper'],
    title='Crypto Forecast',
    labels={'value':'Price',
            'ds': 'Date'}
)

# Adding a slider
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

C:\Users\mymy1\AppData\Local\Temp\ipykernel_25880\1532265748.py:2: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## 4. Consolidated Previous Steps into Functions

In [9]:
def getData(ticker, window, ma_period):
    """
    Grabs price data from a given ticker. Retrieves prices based on the given time window; from now
    to N days ago.  Sets the moving average period for prediction. Returns a preprocessed DF
    formatted for FB Prophet.
    """
    # Time periods
    now = datetime.now()

    # How far back to retrieve tweets
    ago = now - timedelta(days=window)

    # Designating the Ticker
    crypto = yf.Ticker(ticker)

    # Getting price history
    df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")
    
    # Handling missing data from yahoo finance
    df = df.reindex(
        [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
        fill_value=None
    ).fillna(method='ffill')
    
    # Getting the N Day Moving Average and rounding the values
    df['MA'] = df[['Open']].rolling(window=ma_period).mean().apply(lambda x: round(x, 2))

    # Dropping the NaNs
    df.dropna(inplace=True)

    # Formatted for FB Prophet
    df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
    
    return df

In [10]:
def fbpTrainPredict(df, forecast_period):
    """
    Uses FB Prophet and fits to a appropriately formatted DF. Makes a prediction N days into 
    the future based on given forecast period. Returns predicted values as a DF.
    """
    # Setting up prophet
    m = Prophet(
        daily_seasonality=True, 
        yearly_seasonality=True, 
        weekly_seasonality=True
    )
    
    # Fitting to the prices
    m.fit(df[['ds', 'y']])
    
    # Future DF
    future = m.make_future_dataframe(periods=forecast_period)
        
    # Predicting values
    forecast = m.predict(future)

    # Returning a set of predicted values
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [11]:
def visFBP(df, forecast):
    """
    Given two dataframes: before training df and a forecast df, returns
    a visual chart of the predicted values and actual values.
    """
    # Visual DF
    vis_df = df[['ds','Open']].append(forecast).rename(
        columns={'yhat': 'Prediction',
                 'yhat_upper': "Predicted High",
                 'yhat_lower': "Predicted Low"}
    )

    # Visualizing results
    fig = px.line(
        vis_df,
        x='ds',
        y=['Open', 'Prediction', 'Predicted High', 'Predicted Low'],
        title='Crypto Forecast',
        labels={'value':'Price',
                'ds': 'Date'}
    )

    # Adding a slider
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    return fig.show()

### Running the functions

In [12]:
# Getting and Formatting Data
df = getData("BTC-USD", window=365, ma_period=3)

# Training and Predicting Data
forecast = fbpTrainPredict(df, forecast_period=21)

# Visualizing Data
visFBP(df, forecast)

20:02:50 - cmdstanpy - INFO - Chain [1] start processing
20:02:50 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\mymy1\AppData\Local\Temp\ipykernel_25880\2308249826.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## 5. Backtesting
Using vectorization

### Helper Functions

In [13]:
def runningFBP(ticker, window=730, ma_period=5, days_to_train=365, forecast_period=10):
    """
    Runs the facebook prophet model over the provided ticker.  Trains with last N days given 
    by days_to_train.  Forecast N days into the future based on given forecast_period.  Moving average 
    is applied to the dataset based on given ma_period. Returns the root mean squared error and a DF 
    of the actual values and the predicted values for the same day.
    """

    # Getting and Formatting Data
    df = getData(ticker, window=window, ma_period=ma_period)
    
    # DF for the predicted values
    pred_df = pd.DataFrame()

    # Running the model on each day
    for i in tqdm(range(days_to_train, window-forecast_period, forecast_period)):

        # Training and Predicting the last day on the forecast
        forecast = fbpTrainPredict(df[i-days_to_train:i], 
                                   forecast_period=forecast_period).tail(forecast_period)[['ds',
                                                                                           'yhat',
                                                                                           'yhat_lower',
                                                                                           'yhat_upper']]

        # Adding the last day predicted
        pred_df = pred_df.append(forecast, ignore_index=True)
        
    # Combining the predicted df and original df
    comb_df = df[['ds', 'Open']].merge(pred_df, 
                                       on='ds', 
                                       how='outer').sort_values(by='ds')
    
    # Setting the index to the dates
    comb_df.set_index('ds', inplace=True)

    return comb_df

In [14]:
def get_prophet_positions(df, short=True):
    """
    For these positions, buy when actual value is above the upper bound and short 
    when actual value is below lower bound. Otherwise do nothing.
    """
    if df['Open'] >= df['yhat_upper']:
        return 1
    elif df['Open'] <= df['yhat_lower'] and short:
        return -1
    else:
        return 0

In [15]:
def fbpBacktest(df, short=True):
    """
    Performs the final backtest using log returns and the positions function.
    Returns the performance.
    """
    # Getting positions
    df['positions'] = df.apply(lambda x: get_prophet_positions(x, short=short), axis=1)

    # Compensating for lookahead bias
    df['positions'] = df['positions'].shift(1)
    
    # Getting log returns
    df['log_returns'] = df['Open'].apply(np.log).diff()

    # Dropping any Nans
    df.dropna(inplace=True)
    
    # Performing the backtest
    returns = df['positions'] * df['log_returns']

    # Inversing the log returns and getting daily portfolio balance
    performance = returns.cumsum().apply(np.exp)
    
    return performance

### Running Backtesting Functions

In [21]:
# Running the model and getting forecast
bt_df = runningFBP("BTC-USD", 
                   window=730, 
                   ma_period=5, 
                   days_to_train=370, 
                   forecast_period=10)

# Performing the backtest
performance = fbpBacktest(bt_df, short=True)

# Visualizing results
px.line(performance,
        x=performance.index,
        y=performance,
        title='Portfolio Performance',
        labels={"y": "Portfolio Balance",
                "ds": "Date"})

  0%|          | 0/35 [00:00<?, ?it/s]21:46:12 - cmdstanpy - INFO - Chain [1] start processing
21:46:12 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\mymy1\AppData\Local\Temp\ipykernel_25880\2571046977.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

  3%|▎         | 1/35 [00:03<01:46,  3.13s/it]21:46:15 - cmdstanpy - INFO - Chain [1] start processing
21:46:15 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\mymy1\AppData\Local\Temp\ipykernel_25880\2571046977.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

  6%|▌         | 2/35 [00:06<01:42,  3.10s/it]21:46:18 - cmdstanpy - INFO - Chain [1] start processing
21:46:18 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\mymy1\AppData\Local\Temp\ipykernel_25880\2571046977.py:26: FutureWarning:

The frame.append method is deprecated and